In [1]:
import SimpleITK as sitk
from pathlib import Path
from tqdm.auto import tqdm
import glob
import os

/root/projects/MICCAI22_ADN/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


ids = [x.split("/")[-1] for x in glob.glob("../data/dicom-0/*") if (
    x.split("/")[-1].startswith("0") and
    len(x.split("/")[-1]) == 7
)]


test_ids = "0073410, 0072723, 0226290, 0537908, 0538058, 0091415, 0538780, 0073540, 0226188, 0226258, 0226314, 0091507, 0226298, 0538975, 0226257, 0226142, 0072681, 0091538, 0538983, 0537961, 0091646, 0072765, 0226137, 0091621, 0091458, 0021822, 0538319, 0226133, 0091657, 0537925, 0073489, 0538502, 0091476, 0226136, 0538532, 0073312, 0539025, 0226309, 0226307, 0091383, 0021092, 0537990, 0226299, 0073060, 0538505, 0073424, 0091534, 0226125, 0072691, 0538425, 0226199, 0226261"
test_ids = test_ids.split(", ")


train_ids = set(ids) - set(test_ids)

len(test_ids), len(train_ids)

(52, 345)

In [3]:
train_ids

{'0019983',
 '0021023',
 '0021061',
 '0041698',
 '0041699',
 '0072680',
 '0072682',
 '0072688',
 '0072693',
 '0072697',
 '0072729',
 '0072734',
 '0072752',
 '0072762',
 '0072776',
 '0072854',
 '0072864',
 '0072910',
 '0072916',
 '0072921',
 '0072946',
 '0072950',
 '0072961',
 '0072975',
 '0072984',
 '0073000',
 '0073007',
 '0073036',
 '0073048',
 '0073074',
 '0073100',
 '0073112',
 '0073160',
 '0073189',
 '0073207',
 '0073213',
 '0073226',
 '0073264',
 '0073278',
 '0073288',
 '0073299',
 '0073307',
 '0073318',
 '0073326',
 '0073338',
 '0073355',
 '0073366',
 '0073372',
 '0073379',
 '0073397',
 '0073441',
 '0073445',
 '0073465',
 '0073496',
 '0073517',
 '0073521',
 '0073523',
 '0073525',
 '0073526',
 '0073531',
 '0073533',
 '0073542',
 '0073543',
 '0074410',
 '0091366',
 '0091370',
 '0091371',
 '0091373',
 '0091374',
 '0091377',
 '0091379',
 '0091381',
 '0091386',
 '0091387',
 '0091388',
 '0091391',
 '0091392',
 '0091393',
 '0091394',
 '0091398',
 '0091400',
 '0091401',
 '0091402',
 '00

In [29]:


SRC_MASK_DIR = "../data/mask"

TARGET_DIR = "../data/nifti"

for dicom_directory in tqdm(sorted(glob.glob("../data/dicom-0/*/CT"))):
    id = dicom_directory.split("/")[-2]
    target_id_dir = Path(os.path.join(TARGET_DIR, id))
    Path.mkdir(target_id_dir, parents=True, exist_ok=True)

    ct_reader = sitk.ImageSeriesReader()
    dicom_names = ct_reader.GetGDCMSeriesFileNames(dicom_directory)
    ct_reader.SetFileNames(dicom_names)
    ct_image = ct_reader.Execute()

    sitk.WriteImage(ct_image, os.path.join(target_id_dir, f'CT.nii'))

    src_mask_id_dir = os.path.join(SRC_MASK_DIR, id)

    mask_filepaths = sorted(glob.glob(os.path.join(src_mask_id_dir, '*.png')))
    mask_reader = sitk.ImageSeriesReader()
    mask_reader.SetFileNames(mask_filepaths)
    mask_image = mask_reader.Execute()

    mask_image.CopyInformation(ct_image)
    sitk.WriteImage(mask_image, os.path.join(target_id_dir, f'mask.nii'))

  0%|          | 0/397 [00:00<?, ?it/s]

  0%|          | 1/397 [00:00<00:40,  9.67it/s]WARNING: In /tmp/SimpleITK-build/ITK-prefix/include/ITK-5.3/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x55dd38689c50): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.226133

ImageSeriesReader (0x55dd38689c50): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.188488

  1%|          | 3/397 [00:00<01:04,  6.07it/s]WARNING: In /tmp/SimpleITK-build/ITK-prefix/include/ITK-5.3/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x55dd38689c50): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0366154

  3%|▎         | 11/397 [00:01<00:49,  7.79it/s]WARNING: In /tmp/SimpleITK-build/ITK-prefix/include/ITK-5.3/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x55dd38689c50): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0236923

ImageSeriesReader (0x55dd38689c50): Non uniform sampling or missing slices detected,  maximum nonun

In [8]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImage,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
    ResizeWithPadOrCropd
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

print_config()

MONAI version: 1.4.dev2405
Numpy version: 1.26.3
Pytorch version: 2.2.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c6050987c68026b37acdd2c1e4b2f00dc7ea7b44
MONAI __file__: /<username>/projects/MICCAI22_ADN/venv/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.2.0
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.12.0
Pillow version: 10.2.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.66.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.8
pandas version: 2.2.0
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd ver

In [2]:
data_dir = "../data/nifti"


In [9]:
train_images = sorted(glob.glob(os.path.join(data_dir, "*", "CT_stripped.nii")))
train_labels = sorted(glob.glob(os.path.join(data_dir, "*", "mask.nii")))
data_dicts = [{"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)]


In [10]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        # ScaleIntensityRanged(
        #     keys=["image"],
        #     a_min=-57,
        #     a_max=164,
        #     b_min=0.0,
        #     b_max=1.0,
        #     clip=True,
        # ),
        # CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(keys=["image", "label"], pixdim=(1.2, 1.2, 5.0), mode=("bilinear", "nearest")),
        ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=(256, 256, 40)),
        # RandCropByPosNegLabeld(
        #     keys=["image", "label"],
        #     label_key="label",
        #     spatial_size=(96, 96, 96),
        #     pos=1,
        #     neg=1,
        #     num_samples=4,
        #     image_key="image",
        #     image_threshold=0,
        # ),
        # user can also add other random transforms
        # RandAffined(
        #     keys=['image', 'label'],
        #     mode=('bilinear', 'nearest'),
        #     prob=1.0, spatial_size=(96, 96, 96),
        #     rotate_range=(0, 0, np.pi/15),
        #     scale_range=(0.1, 0.1, 0.1)),
    # SaveImaged(keys=['image', 'label'], )
    ]
)

In [6]:
check_ds = Dataset(data=data_dicts, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
# check_data = first(check_loader)


In [39]:
saver = SaveImage(output_ext='.nii')
# saver(check_ds[0]['image'], filename='test')

In [30]:
data.keys()

dict_keys(['image', 'label'])

In [36]:
from tqdm import tqdm
from pathlib import Path

In [40]:

for data in tqdm(check_ds):
    image = data['image']
    label = data['label']
    id = image.meta['filename_or_obj'].split("/")[-2]
    Path(f"SOMEFOLDER/{id}").mkdir(parents=True, exist_ok=True)
    saver(image, filename=f"SOMEFOLDER/{id}/CT")
    saver(label, filename=f"SOMEFOLDER/{id}/mask")

  0%|          | 0/397 [00:00<?, ?it/s]

2024-02-05 03:08:20,595 INFO image_writer.py:197 - writing: SOMEFOLDER/0019983/CT.nii
2024-02-05 03:08:20,660 INFO image_writer.py:197 - writing: SOMEFOLDER/0019983/mask.nii


  0%|          | 1/397 [00:00<02:01,  3.25it/s]

2024-02-05 03:08:20,869 INFO image_writer.py:197 - writing: SOMEFOLDER/0021023/CT.nii
2024-02-05 03:08:20,925 INFO image_writer.py:197 - writing: SOMEFOLDER/0021023/mask.nii


  1%|          | 2/397 [00:00<01:57,  3.37it/s]

...

 99%|█████████▉| 395/397 [01:53<00:00,  3.80it/s]

2024-02-05 03:10:14,316 INFO image_writer.py:197 - writing: SOMEFOLDER/0539035/CT.nii
2024-02-05 03:10:14,347 INFO image_writer.py:197 - writing: SOMEFOLDER/0539035/mask.nii


100%|█████████▉| 396/397 [01:53<00:00,  4.25it/s]

2024-02-05 03:10:14,499 INFO image_writer.py:197 - writing: SOMEFOLDER/0539043/CT.nii
2024-02-05 03:10:14,531 INFO image_writer.py:197 - writing: SOMEFOLDER/0539043/mask.nii


100%|██████████| 397/397 [01:54<00:00,  3.48it/s]


In [28]:
data_dicts = loader(data_dicts)

FileNotFoundError: [Errno 2] No such file or directory: 'label'

In [23]:
len(data_dicts)

4

In [20]:
ensure_channel_first = EnsureChannelFirstd(keys=["image", "label"])
datac_dict = ensure_channel_first(data_dict)
print(f"image shape: {datac_dict['image'].shape}")

NameError: name 'data_dict' is not defined